In [142]:
import pandas as pd
from datetime import datetime, timedelta
from ast import literal_eval
rest2 = pd.read_csv('kaggle_task.csv')
rest1 = pd.read_csv('main_task.csv')

In [143]:
rest1['sample'] = 1
rest2['sample'] = 0
rest2['Rating'] = 0
rest = rest2.append(rest1, sort = False).reset_index(drop = True)

# Расшариваем кухни

In [144]:
# Заменяем пропущенные значения на '[propusk]'
rest['Cuisine Style'] = rest['Cuisine Style'].fillna('["propusk"]')

# Делаем общий set из кухонь
kuhni = set()  
for i in rest['Cuisine Style']: 
    for j in literal_eval(i):
        kuhni.add(j)

# Используемая функция, возвращающая 1 если эта кухня есть
def estornot(s):
    if kuh in s:
        return 1
    return 0

# Делаем новые столбцы, используя функцию estornot
for kuh in kuhni:
    rest[kuh] = rest['Cuisine Style'].apply(estornot)    

# Заменяем описание кухонь на количество
def lencuz(s):
    if literal_eval(s) == ["propusk"]:
        return 0
    return len(literal_eval(s))
rest['Cuisine Style'] = rest['Cuisine Style'].apply(lencuz)

# Расшариваем ценник

In [145]:
# заменяем пропущенные значения на пропуск
def prnarange(s):
    if pd.isna(s) == True:
        return 'napri'
    return s
rest['Price Range'] = rest['Price Range'].apply(prnarange)

# Функция: заменяем доллары на евро
def doleur(s):
    if s == '$':
        return 'a'
    if s == '$$ - $$$':
        return 'bb'
    if s == '$$$$':
        return 'ccc'
    return s
rest['Price Range'] = rest['Price Range'].apply(doleur)

# Делаем общий set из ценников
zennik = set()  
for i in rest['Price Range']: 
    zennik.add(i)
    
# Используемая функция, возвращающая 1 если этот ценник есть
def estornotzen(s):
    if zen in s:
        return 1
    return 0  
# Делаем новые столбцы, используя функцию estornot
for zen in zennik:
    rest[zen] = rest['Price Range'].apply(estornotzen)
    
# Заменяем ценники числовым эквивалентом
def prrangelen(s):
    if s == 'napri':
        return 0
    return len(s)
rest['Price Range'] = rest['Price Range'].apply(prrangelen)

# Расшариваем города

In [146]:
# Делаем список городов
spisokgorodov = set()
for i in rest['City']:
    spisokgorodov.add(i)

# Используемая функция, возвращающая 1 если ресторан в городе s
def spisgor(s):
    if gor in s:
        return 1
    return 0

# Делаем новые столбцы, используя функцию spisgor
for gor in spisokgorodov:
    rest[gor] = rest['City'].apply(spisgor)

# Расшариваем страны

In [147]:
# Делаем вручную словарь страна: город
countryes = {'Italy': ['Rome', 'Milan'], 'Germany': ['Berlin', 'Munich', 'Hamburg'], 'Slovenia': ['Ljubljana'], 'Holland': ['Amsterdam'], 'Sweden': ['Stockholm'], 'Denmark': ['Copenhagen'], 'Czech': ['Prague'], 'Switz': ['Geneva', 'Zurich'], 'Luxembourg': ['Luxembourg'], 'Portugal': ['Lisbon', 'Oporto'], 'Scotland': ['Edinburgh'], 'Austria': ['Vienna'], 'England': ['London'],'Belgium': ['Brussels'], 'Poland': ['Warsaw', 'Krakow'], 'France': ['Paris', 'Lyon'], 'Ireland': ['Dublin'], 'Finland': ['Helsinki'], 'Greece': ['Athens'], 'Spain': ['Madrid', 'Barcelona'], 'Norway': ['Oslo'], 'Romania': ['Budapest'], 'Slovakia': ['Bratislava']}

# Делаем новый столбец со странами
def countre(s):
    for i in countryes:
        if s in countryes[i]:
            return i
rest['Country'] = rest['City'].apply(countre)

# Делаем список стран
spisokstran = set()
for i in rest['Country']:
    spisokstran.add(i)

# Используемая функция, возвращающая 1 если страна есть s
def spisstr(s):
    if stra in s:
        return 1
    return 0
for stra in spisokstran:
    rest[stra] = rest['Country'].apply(spisstr)

# Расшариваем ['Ranking']

In [148]:
#Делаем функцию делящую лист на равные части
def split(arr, size):
    arrs = []
    while len(arr) > size:
        pice = arr[:size]
        arrs.append(pice)
        arr = arr[size:]
    arrs.append(arr)
    return arrs

#Делим на равные части
asde = split(sorted(rest['Ranking']), 4000)

 #Делаем функцию, возвращающую 1, если значение естьв промежутке
def parsrating(s):
    if asde[i][0] <= s <= asde[i][-1]:
        return 1
    return 0

 #Применяем функцию
for i in range(10):
    rest[i] = rest['Ranking'].apply(parsrating)

# Усредняем Ranking

In [149]:
city_mean = rest.groupby(['City'])['Ranking'].mean()
city_count = rest['City'].value_counts(ascending=False)
rest['ratingmeancity'] = rest['City'].apply(lambda x: city_mean[x])
rest['ratingvccity'] = rest['City'].apply(lambda x: city_count[x])
rest['newranking'] = (rest['Ranking'] - rest['ratingmeancity']) / rest['ratingvccity']

# Есть отзывы о ресторане или нет

In [150]:
# Функция, возвращает 1 если есть, 0 если нет
def otzestornot(s):
    if s == '[[], []]':
        return 0
    return 1

# Добавляем столбцы
rest['otzestornott'] = rest['Reviews'].apply(otzestornot)

# Разность дат отзывов

In [151]:
# Используемая функция
def raznotz(se):
    s = literal_eval(rest['Reviews'][1])[1][0]
    s1 = literal_eval(rest['Reviews'][1])[1][1]
    dt2 = datetime.strptime(s, '%m/%d/%Y') - datetime.strptime(s1, '%m/%d/%Y')
    if se == '[[], []]':
        return 116
    else:
        try:
            aw = datetime.strptime(literal_eval(se)[1][0], '%m/%d/%Y') -  datetime.strptime(literal_eval(se)[1][1], '%m/%d/%Y')
            return abs(aw.days)
        except:
            pass

# Применяем функцию
rest['razndatotz'] = rest['Reviews'].apply(raznotz)
# Заменяем пропущенные значения медианой
rest['razndatotz'] = rest['razndatotz'].fillna(102)

# Выбираем столичные рестораны

In [152]:
# Вручную делаем список нестоличных городов
nestol = ['Munich', 'Oporto', 'Milan', 'Barcelona', 'Zurich', 'Lyon', 'Hamburg', 'Geneva', 'Krakow']

# Используемая функция
def stol(s):
    if s in nestol:
        return 0
    else:
        return 1

# Применяем функцию
rest['Stolichniy'] = rest['City'].apply(stol)

# Находится ли ресторан в западной Европе

In [153]:
# делаем список городов восточной Европы
vosteur = ['Bratislava', 'Warsaw', 'Budapest', 'Prague', 'Ljubljana', 'Krakow']

# Ипользуемая функция
def vosteuropa(s):
    if s in vosteur:
        return 0
    else:
        return 1
    
# Применяем функцию
rest['ZEur'] = rest['City'].apply(vosteuropa)

### Находится ли ресторан в туристической Мекке (0 - нет, 1 - да) (находится ли ресторан в 25 самых посещаемых городах Европы, исходя из рейтинга tripadvisor - https://www.tripadvisor.ru/TravelersChoice-Destinations-cTop-g4)

In [154]:
# делаем список из городов, входящих в рейтинг.
gorMek = ['Paris', 'London', 'Rome', 'Barcelona', 'Prague', 'Lisbon', 'Amsterdam', 'Budapest', 'Berlin', 'Edinburgh', 'Vienna', 'Krakow']

# Используемая функция
def Mek(s):
    if s in gorMek:
        return 1
    else:
        return 0

# Добавляем столбец
rest['Mekka'] = rest['City'].apply(Mek)

# Топ по количеству ресторанов на пятерочку

In [155]:
# Делаем список городов на пятерочку
citys5 = ['Milan', 'Berlin', 'Madrid', 'Paris', 'London', 'Barcelona', 'Rome']

# Используемая функция
def kolrest(s):
    if s in citys5:
        return 1
    else:
        return 0

# Применяем функцию
rest['City5'] = rest['City'].apply(kolrest)

# Сколько кухонь в ресторане

In [156]:
# Ресторан с одной кухней
def onequz(s):
    if s == 1:
        return 1
    else:
        return 0

# Добавляем столбец
rest['OneQuz'] = rest['Cuisine Style'].apply(onequz)

# Ресторан с двумя кухнями
def twoquz(s):
    if s == 2:
        return 1
    else:
        return 0
# Добавляем столбец    
rest['TwoQuz'] = rest['Cuisine Style'].apply(twoquz)

# Ресторан с тремя и более кухнями
def triquz(s):
    if s >= 3:
        return 1
    else:
        return 0

# Добавляем столбец
rest['TriQuz'] = rest['Cuisine Style'].apply(triquz)

# Меняем название города на его население

In [157]:
naselenie = {'Amsterdam': 859732,
 'Athens': 655780,
 'Barcelona': 1620343,
 'Berlin': 3601131,
 'Bratislava': 424428,
 'Brussels': 1198726,
 'Budapest': 1749734,
 'Copenhagen': 615993,
 'Dublin': 553165,
 'Edinburgh': 482005,
 'Geneva': 495249,
 'Hamburg': 1830584,
 'Helsinki': 643272,
 'Krakow': 766739,
 'Lisbon': 505526,
 'Ljubljana': 279631,
 'London': 8787892,
 'Luxembourg': 602005,
 'Lyon': 515695,
 'Madrid': 3223334,
 'Milan': 1366180,
 'Munich': 1456039,
 'Oporto': 214349,
 'Oslo': 673469,
 'Paris': 2140526,
 'Prague': 1280508,
 'Rome': 2872800,
 'Stockholm': 961609,
 'Vienna': 1840573,
 'Warsaw': 1758143,
 'Zurich': 402762}
def nasel(s):
    for i in naselenie:
        if s == i:
            return naselenie[i]
rest['City'] = rest['City'].apply(nasel)

# Город миллионник

In [158]:
rest['1000000'] = rest['City'].apply(lambda x: 1 if x > 1000000 else 0)
rest['City'] = rest['City'].apply(lambda x: x / 1000000)

## Есть ли пропуски в ['Number of Reviews']

In [159]:
rest['Number_of_Reviews_isNAN'] = pd.isna(rest['Number of Reviews']).astype('uint8')
# ставим ноль
rest['Number of Reviews'] = rest['Number of Reviews'].fillna(0)

In [160]:
rest = pd.get_dummies(rest, columns=['Cuisine Style'], dummy_na=True)

In [161]:
# Удаляем объекты
rest = rest.drop(['Country', 'Name' ,'Reviews', 'URL_TA', 'ID_TA'], axis = 1)

In [162]:
import numpy as np
def round_half(num):
    if num < 1.25:
        num = 1.0
    elif num < 1.75:
        num = 1.5
    elif num < 2.25:
        num = 2
    elif num < 2.75:
        num = 2.5
    elif num < 3.25:
        num = 3.0
    elif num < 3.75:
        num = 3.5
    elif num < 4.25:
        num = 4.0
    elif num < 4.75:
        num = 4.5
    else:
        num = 5.0
    return num

vfunc = np.vectorize(round_half)

In [163]:
rest4 = rest.query('sample == 1').drop(['sample'], axis=1)

In [164]:
X = rest4.drop(['Restaurant_id', 'Rating'], axis = 1)  
y = rest4['Rating']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=250, max_depth=30, min_samples_split=4, min_samples_leaf=5, n_jobs=-1, verbose=1)  
  
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
  
# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  1.5min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s


MAE: 0.2074472157991995


[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:    0.5s finished


```
test_data = rest.drop(['Restaurant_id'], axis = 1)
test_data = test_data.query('sample == 0').drop(['sample'], axis=1)
test_data = test_data.drop(['Rating'], axis=1)
predict_submission = regr.predict(test_data)
predict_submission = vfunc(predict_submission)
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head()
```